In [1]:
"""For submitting workflows for perovskites"""

#NOTE: user should change this to their own launchpad path
# lpad_file_path = '/global/homes/d/dbroberg/atomate_fworkers/my_launchpad.yaml'
# lpad_file_path = '/Users/dpbroberg/bin/my_launchpad.yaml'
lpad_file_path = '/home/yig319/Dropbox/atomate' # have to be /my_launchpad.yaml, from VASP?

In [1]:
import os

from pymatgen.core import Composition  #Element, Structure, Lattice
from pymatgen.io.vasp import Poscar, Outcar

from fireworks import Workflow
from fireworks.core.launchpad import LaunchPad

from structure import PerfectPerovskite, StrainedPerovskite

from pymatgen.io.vasp.sets import MPRelaxSet

from atomate.vasp.fireworks.core import OptimizeFW
from atomate.vasp.workflows.base.ferroelectric import get_wf_ferroelectric

from monty.serialization import dumpfn, loadfn



In [2]:
"""Lattice constant workflow related"""

def generate_lattconst_wf( list_elt_sets, functional='PBE', vasp_cmd = '>>vasp_cmd<<',
                           db_file = '>>db_file<<', submit=False, scan_smart_lattice=None):
    """Generates a workflow which calculates lattice constants
    through optimization fireworks for a given functional type

    NOTE: that the SCAN functionality might be reliant on some Custodian features from Danny's Custodian
    """
    
    if functional in ['PBE', 'LDA']: #Perdew–Burke-Ernzerhof (PBE); LDA: Local Density Approximation
        job_type = 'double_relaxation_run'
        potcar_type = functional
        incar_settings = {"ADDGRID": True, 'EDIFF': 1e-8}
        
    elif functional in ['SCAN']:
        job_type = 'metagga_opt_run'
        potcar_type = 'PBE' #this is the POTCAR that needs to be used for SCAN...
        incar_settings = {'EDIFF': 1e-8, 'ISIF': 7}
        if scan_smart_lattice is None:
            raise ValueError("Need to provide a smarter starting point "
                             "for SCAN lattice constants...")
    # Think PBE, LDA as different methods of DFT calculating, SCAN is obtain lattconst before calculating.
    
    
    fws = []

    for elt_set in list_elt_sets:
        if functional == 'SCAN':
            # give amount of every element from ABO3
            compkey = Composition({elt_set[0]: 1, elt_set[1]: 1, elt_set[2]: 3})
            # decide lattice constant depend on composition
            lattconst = scan_smart_lattice[compkey]  
        fw = OptimizeFW(s, name="{} {} structure optimization".format(s.composition.reduced_formula, functional),
                        vasp_input_set=vis, vasp_cmd=vasp_cmd, db_file=db_file,
                        job_type=job_type, auto_npar=">>auto_npar<<")
        else:
            lattconst = None
        pp = PerfectPerovskite( Asite=elt_set[0], Bsite=elt_set[1], Osite=elt_set[2],
                                lattconst = lattconst) # obtain original perovskite structure
        s = pp.get_111_struct()

        vis = MPRelaxSet( s, user_incar_settings=incar_settings, potcar_functional=potcar_type) 
        # Implementation of VaspInputSet utilizing parameters in the public Materials Project. 

        fw = OptimizeFW(s, name="{} {} structure optimization".format(s.composition.reduced_formula, functional),
                        vasp_input_set=vis, vasp_cmd=vasp_cmd, db_file=db_file,
                        job_type=job_type, auto_npar=">>auto_npar<<")
        '''A Firework is a workflow step and might be contain several Firetasks.
           Optimize the given structure.'''
        
#     Docstring:  A Firework is a workflow step and might be contain several Firetasks.
#     Init docstring: Optimize the given structure.
        
#     OptimizeFW( structure,
#                 name='structure optimization',
#                 vasp_input_set=None,
#                 vasp_cmd='>>vasp_cmd<<',
#                 override_default_vasp_params=None,
#                 ediffg=None,
#                 db_file='>>db_file<<',
#                 force_gamma=True,
#                 job_type='double_relaxation_run',
#                 max_force_threshold=0.25,
#                 auto_npar='>>auto_npar<<'17:00,
#                 half_kpts_first_relax=False,
#                 parents=None,
#                 **kwargs,)


#     Args:
#         structure (Structure): Input structure.
        
#         name (str): Name for the Firework.
        
#         vasp_input_set (VaspInputSet): input set to use. Defaults to MPRelaxSet() if None.
        
#         vasp_cmd (str): Command to run vasp.
    
#         override_default_vasp_params (dict): If this is not None, these params are passed to 
#             the default vasp_input_set, i.e., MPRelaxSet. This allows one to easily override 
#             some settings, e.g., user_incar_settings, etc.

#         ediffg (float): Shortcut to set ediffg in certain jobs
        
#         db_file (str): Path to file specifying db credentials to place output parsing.
        
#         force_gamma (bool): Force gamma centered kpoint generation
        
#         job_type (str): custodian job type (default "double_relaxation_run")
        
#         max_force_threshold (float): max force on a site allowed at end; otherwise, reject job
        
#         auto_npar (bool or str): whether to set auto_npar. defaults to env_chk: ">>auto_npar<<"
        
#         half_kpts_first_relax (bool): whether to use half the kpoints for the first relaxation
        
#         parents ([Firework]): Parents of this particular Firework.
 
        fws.append( fw) # a list of fireworks list for different fireworks set

    wf = Workflow( fws, name='{} latt const workflow'.format( functional))
    # A Workflow connects a group of FireWorks in an execution order.
    if submit:
        print('Submitting workflow with {} fws for {}'.format( len(list_elt_sets), functional))
        lpad = LaunchPad().from_file(lpad_file_path)
        lpad.add_wf( wf)
    else:
        print('Workflow created with {} fws for {}'.format( len(list_elt_sets), functional))
        return wf

In [3]:
def parse_wf_for_latt_constants( wf_id):
    lpad = LaunchPad().from_file(lpad_file_path)
    # what type of the file we get from from_file ?????????
    
        
    
    
    
    
    
    


    wf = lpad.get_wf_by_fw_id( wf_id)

    lattdata = {}
    print('{} workflow retrieved with {} fws in it'.format( wf.name, len(wf.fws)))
    for fw in wf.fws:
        print('\t{}'.format( fw.name))
        if 'structure optimization' not in fw.name:
            raise ValueError("Not a recognized firework!")
        elif fw.state != 'COMPLETED':
            print('\t\tstatus = {}, so skipping'.format( fw.state))
            continue

        pat = fw.launches[-1].launch_dir # fw's launch directory
        s = Poscar.from_file( os.path.join( pat, 'CONTCAR.relax2.gz')).structure # Reads a Poscar from a launch_dir.

    '''Poscar(  structure,
                comment=None,
                selective_dynamics=None,
                true_names=True,
                velocities=None,
                predictor_corrector=None,
                predictor_corrector_preamble=None)

        Docstring:     
        Object for representing the data in a POSCAR or CONTCAR file.
        Please note that this current implementation. Most attributes can be set
        directly.

        Args:
            structure (Structure):  Structure object.
            comment (str): Optional comment line for POSCAR. Defaults to unit
                cell formula of structure. Defaults to None.
            selective_dynamics (Nx3 array): bool values for selective dynamics,
                where N is number of sites. Defaults to None.
            true_names (bool): Set to False is the names in the POSCAR are not
                well-defined and ambiguous. This situation arises commonly in
                vasp < 5 where the POSCAR sometimes does not contain element
                symbols. Defaults to True.
            velocities (Nx3 array): Velocities for the POSCAR. Typically parsed
                in MD runs or can be used to initialize velocities.
            predictor_corrector (Nx3 array): Predictor corrector for the POSCAR.
                Typically parsed in MD runs. '''

        nom = s.composition.reduced_formula
        if nom in lattdata: 
            raise ValueError("{} already exists in lattdata??".format( nom))# avoid lattdata repeated add
        elif (max(s.lattice.abc) - min(s.lattice.abc)) > 0.00001 or  (max(s.lattice.angles) - min(s.lattice.angles)) > 0.00001:
            raise ValueError("Error occured with lattice relaxation??".format( s.lattice)) 
            # Make sure lengths of the lattice vectors are not changed by lattice relaxation too much
             
        else:
            lattdata.update( {nom: s.lattice.abc[0]})

    print('\nFinalized lattice constant set:\n{}'.format( lattdata))

    return lattdata


"""LCALCEPs workflow testing related"""

'LCALCEPs workflow testing related'

In [17]:
aoa = {}
aoa.update({'a':'a'})

In [18]:
print(aoa)

{'a': 'a'}


In [4]:
# Core function for create a firework which can be submit to launchpad.

def polarization_wf( polar_structure, nonpolar_structure, submit=False, nimages=8,
                     user_incar_settings = {}, tags = []):
    """

    :param polar_structure: structure of polar structure
    :param nonpolar_structure: structure of nonpolar structure
    :param submit: boolean for submitting
    :param tags: list of string tags
    :return:
    """

    if polar_structure.species != nonpolar_structure.species:
        raise ValueError("WRONG ORDER OF SPECIES: {} vs {}".format( polar_structure.species, nonpolar_structure.species))

    vasp_input_set_params = {'user_incar_settings': user_incar_settings} # user incar setting ???
    
    
    
    
    
    
    
    
    
    
    
    
    wf = get_wf_ferroelectric( polar_structure, nonpolar_structure, vasp_cmd=">>vasp_cmd<<",
                              db_file='>>db_file<<', vasp_input_set_polar="MPStaticSet",
                              vasp_input_set_nonpolar="MPStaticSet", relax=False,
                              vasp_relax_input_set_polar=vasp_input_set_params,
                              vasp_relax_input_set_nonpolar=vasp_input_set_params,
                              nimages=nimages, hse=False, add_analysis_task=True, tags=tags)
    '''Returns a workflow to calculate the spontaneous polarization of polar_structure using
    a nonpolar reference phase structure and linear interpolations between the polar and
    nonpolar structure.'''

    print('workflow created with {} fws'.format( len(wf.fws)))

    if submit:
        print("\tSubmitting Polarization workflow")
        lp = LaunchPad().from_file( lpad_file_path)
        lp.add_wf(wf) # add wf to launchpad list
    else:
        return wf

In [5]:
def get_wf_timing( wf_id, returnval = False):

    lp = LaunchPad().from_file(lpad_file_path)
    wf = lp.get_wf_by_fw_id( wf_id) 
    out_run_stats = []
    just_non_polar_stats = []
    for fw in wf.fws:
        ld = fw.launches[-1].launch_dir
        out = None
        if 'OUTCAR' in os.listdir(ld):
            out = Outcar( os.path.join( ld, 'OUTCAR')) 
        elif 'OUTCAR.gz' in os.listdir(ld):
            out = Outcar( os.path.join( ld, 'OUTCAR.gz'))
        if out:
            out_run_stats.append( out.run_stats.copy())
            if 'nonpolar_polarization' in fw.name:
                just_non_polar_stats.append( out.run_stats.copy())
        ld += '/polarization'
        if os.path.exists( ld):
            out = None
            if 'OUTCAR' in os.listdir(ld):
                out = Outcar( os.path.join( ld, 'OUTCAR'))
            elif 'OUTCAR.gz' in os.listdir(ld):
                out = Outcar( os.path.join( ld, 'OUTCAR.gz'))
            if out:
                out_run_stats.append( out.run_stats.copy())
    cores = out_run_stats[0]['cores']
    print('Workflow {} retrieved {} Outcars all run on {} cores'.format( wf.name, len(out_run_stats), cores))
    timestat = {k: 0 for k in ['Elapsed time (sec)', 'System time (sec)',
                               'User time (sec)', 'Total CPU time used (sec)']}
    print('\nNon-Polar calc (non-polarization) alone took:')
    if len( just_non_polar_stats) != 1:
        raise ValueError("Too many non polar calcs?? = {}".format( len(just_non_polar_stats)))
    else:
        for k,v in just_non_polar_stats[0].items():
            if k in timestat.keys():
                print("\t{}: {}".format( k, round(v, 2)))

    for out in out_run_stats:
        if out['cores'] != cores:
            raise ValueError("Inconsisten number of cores for timing! {} vs {}".format( cores, out['cores']))
        for k,v in out.items():
            if k in timestat:
                timestat[k] += v

    print("\nSummary of TOTAL wf timing:")
    for k,v in timestat.items():
        print("\t{}: {}".format( k, round(v, 2)))
    if not returnval:
        return
    else:
        return {'tot': timestat, 'nonpolar': just_non_polar_stats}

"""Perturbation workflow related"""

'Perturbation workflow related'

In [6]:
def perturb_wf_setup( perovskite, structure_type='111',
                      Nstruct = 100, perturbamnt=None, max_strain=0.06,
                      nimages = 8, tags = []):
# This function need to input "lattconst.json" and load file from: "lpad_file_path"
    
        if perturbamnt is None:
            perturbamnt = perovskite.lattconst * 0.04    # limit the perturbation

        print("Setting up {} different perturbation polarization approaches\nMax strain = {}, "
              "Perturbation amount = {}".format( Nstruct, max_strain, perturbamnt))

        allowed_struct_type = ['111', '211', 's2s21', 's2s22'] 
        if structure_type not in allowed_struct_type:
            raise ValueError("{} not in {}".format( structure_type,
                                                    allowed_struct_type))

        fws = []
        pert_N_structs = [perovskite.get_struct_from_structure_type( structure_type).as_dict()]
        # save structure type as dictionary form
        
        user_incar_settings = {"ADDGRID": True, 'EDIFF': 1e-8, "NELMIN": 6}
        for nind in range(Nstruct):
            sclass = PerfectPerovskite( Asite= perovskite.eltA, Bsite= perovskite.eltB, Osite= perovskite.eltC,
                                        lattconst=perovskite.lattconst ) # define structure without strained
            strain_class = StrainedPerovskite.generate_random_strain( sclass, structure_type=structure_type,
                                                                     max_strain=max_strain, perturb_amnt=perturbamnt)
            #define strained structure
            
            tmp_wf = polarization_wf( strain_class.structure, strain_class.base, submit=False, nimages=nimages,
                                      user_incar_settings=user_incar_settings, tags = tags)
            fws.extend( tmp_wf.fws) # extend fws with new wf Nstruct times
            pert_N_structs.append( strain_class.structure.as_dict()) 

        print("Submitting Polarization workflow with {} fireworks".format( len(fws)))
        wf = Workflow( fws)
        lp = LaunchPad().from_file( lpad_file_path)    #The LaunchPad manages the FireWorks database.
        
        lp.add_wf(wf)   # add wf to launchpad

In [7]:
if __name__ == "__main__":
    init_list = [['Sr', 'Ti', 'O'],
                 ['Ca', 'Ti', 'O'],
                 ['Pb', 'Ti', 'O'],
                 ['Ba', 'Ti', 'O'],
                 ['Sn', 'Ti', 'O'],
                 ['Sr', 'Zr', 'O'],
                 ['Ca', 'Zr', 'O'],
                 ['Pb', 'Zr', 'O'],
                 ['Ba', 'Zr', 'O'],
                 ['Sn', 'Zr', 'O'],
                 ['Li', 'Nb', 'O'],
                 ['La', 'Al', 'O'],
                 ['Li', 'Ta', 'O'],
                 ['La', 'Ta', 'O'],
                 ['Y', 'Al', 'O'],
                 ['Gd', 'Sc', 'O'],
                 ['Dy', 'Sc', 'O'],
                 ['Nd', 'Sc', 'O'],
                 ['Sm', 'Sc', 'O'],
                 ['La', 'Lu', 'O']]

    """Lattice constant generation related"""
    for func in ['PBE', 'LDA']:
        generate_lattconst_wf(init_list, functional=func, submit=True)

    gga_latt_dict = parse_wf_for_latt_constants( 3257) # input wf_id, return lattice constant to dict
    generate_lattconst_wf([init_list[0]], functional='SCAN', submit=True,
                          scan_smart_lattice=gga_latt_dict) # SCAN mode returns wf with fws
    gga_latt_dict = parse_wf_for_latt_constants( 3301)
    lda_latt_dict = parse_wf_for_latt_constants( 3321)
    from monty.serialization import dumpfn
    from monty.json import MontyEncoder
    
    dumpfn( {'gga': gga_latt_dict, 'lda': lda_latt_dict}, 'latt_consts.json', cls=MontyEncoder)
    # save to json file

    """Polarization testing related"""
    #first test on a cubic material (PbTiO3)
    
    from pymatgen import MPRester, Structure
    with MPRester() as mp:
        s = mp.get_structure_by_material_id('mp-19845') # return structure from mp
    pert_coords = []
    for site in s.sites:
        if site.specie.symbol == 'Ti':
            pert_coords.append( site.coords + np.array( [0., 0., 0.25])) # why plus 0.25 for Ti ???
            
            
            
            
            
            
            
            
        else:
            pert_coords.append( site.coords)
    pert_struct = Structure( s.lattice, s.species, pert_coords, coords_are_cartesian=True)

    # input original structure and perturbated one, return wf and submit it
    polarization_wf(s, pert_struct, submit=True, wfid="TestPbTiO3") 
    polarization_wf(s, pert_struct, submit=True, wfid="BasicTessTestPbTiO3x2")

    
    #second test on a tetragonal (known polar) material (PbTiO3)
    #recreate this arxiv paper's result: https://arxiv.org/pdf/1702.04817.pdf
    #teragonal cell:  a = 3.844 , c/a = 1.240,  volume = 70.4 , Displacement of Ti atom = 0.058 * c
    #resulting polarization = 125.5 (μC/cm2)
    from pymatgen.core import Lattice, Element, Structure
    lattice = Lattice([[3.844, 0., 0.], [0., 3.844, 0.], [0., 0., 1.24 * 3.844]])
    species = [Element("Pb"), Element("Ti"), Element("O"), Element("O"), Element("O")]
    coords = [[0., 0., 0.], [0.5, 0.5, 0.5], [0.5, 0.5, 0.],
              [0.5, 0., 0.5], [0., 0.5, 0.5]]
    perfect_struct = Structure( lattice, species, coords, coords_are_cartesian=False)
    # create perfect structure with defined lattice, species, coords.
    
    pert_coords = perfect_struct.cart_coords[:]
    pert_coords[1] += np.array([0., 0., 1.24 * 3.844 * 0.058]) # add perturb coords
    pert_struct = Structure( lattice, species, pert_coords, coords_are_cartesian=True)
    
    polarization_wf(perfect_struct, pert_struct, submit=False, wfid="TestTetragonalPbTiO3")
    # perturb wf without submitting
    
    
    #TRY above test again... with MPRester on
    from pymatgen import MPRester, Structure
    with MPRester() as mp:
        tmp_perfect_struct = mp.get_structure_by_material_id('mp-19845')
        pert_struct = mp.get_structure_by_material_id('mp-20459')
    # reorder perfect structure sites as Ti, Pb, O, O, O
    perfect_struct = Structure( tmp_perfect_struct.lattice,
                                [tmp_perfect_struct.species[ind] for ind in [3, 4, 0, 1, 2]],
                                [tmp_perfect_struct.cart_coords[ind] for ind in [3, 4, 0, 1, 2]],
                                coords_are_cartesian=True)
    polarization_wf(perfect_struct, pert_struct, submit=False, wfid="Test3TetragonalPbTiO3")
    
    
    
    # test on several randomly generated structures (cubic PbTiO3 ) to test timing
    from pymatgen import MPRester
    with MPRester() as mp:
        s = mp.get_structure_by_material_id('mp-19845')
    bpc = PerfectPerovskite( Asite="Pb", Bsite="Ti", Osite="O", lattconst=s.lattice.abc[0])
    for struct_type in ['111', '211', 's2s21', 's2s22']:
        print('\n-> Create workflow for {}'.format( struct_type))
        sp_class = StrainedPerovskite.generate_random_strain(bpc, structure_type=struct_type,
                                                             max_strain=0.06, perturb_amnt=None)
        polarization_wf( sp_class.base, sp_class.structure, submit = False)

    get_wf_timing( 3809) #get timing for 111 PbTiO3 case

    #get timing for test distortions with 111, 211, s2s21, s2s22
    outset = {}
    for wf_id in [3823, 3830, 3837, 3844]:
        print("----> Doing {}".format(wf_id))
        out = get_wf_timing( wf_id, returnval=True)
        outset[wf_id] = out
    print('\n------\n',outset)

    """Polarization workflow generation related"""

No lattice constant given. Using 4.01 for SrTiO3
No lattice constant given. Using 4.01 for CaTiO3
No lattice constant given. Using 4.01 for PbTiO3
No lattice constant given. Using 4.01 for BaTiO3
No lattice constant given. Using 4.01 for SnTiO3
No lattice constant given. Using 4.24 for SrZrO3
No lattice constant given. Using 4.24 for CaZrO3
No lattice constant given. Using 4.24 for PbZrO3
No lattice constant given. Using 4.24 for BaZrO3
No lattice constant given. Using 4.24 for SnZrO3
No lattice constant given. Using 4.16 for LiNbO3
No lattice constant given. Using 3.87 for LaAlO3
No lattice constant given. Using 4.16 for LiTaO3
No lattice constant given. Using 4.16 for LaTaO3
No lattice constant given. Using 3.87 for YAlO3
No lattice constant given. Using 4.29 for GdScO3
No lattice constant given. Using 4.29 for DyScO3
No lattice constant given. Using 4.29 for NdScO3
No lattice constant given. Using 4.29 for SmScO3
No lattice constant given. Using 4.522 for LaLuO3
Submitting workflow 

NameError: name 'lpad_file_path' is not defined

In [31]:
    #test with random distortions with above chemistry
    latt_consts = loadfn( 'latt_consts.json')['gga']
    for scomp in [init_list[0]]:
        print(scomp)
        skey = ''.join(scomp) + '3'
        if skey not in latt_consts:
            raise ValueError("{} is not in the lattice constants dictionary!".format(skey))
        perovskite = PerfectPerovskite( Asite= scomp[0], Bsite= scomp[1], Osite= scomp[2],
                                        lattconst=latt_consts[skey])
        perturb_wf_setup(perovskite, structure_type='111',
                         Nstruct=2, perturbamnt=None, max_strain=0.06,
                         nimages=5, tags=['danny_test_polar'])

FileNotFoundError: [Errno 2] No such file or directory: 'latt_consts.json'